In [ ]:
# Install the required packages
%pip install -r requirements.txt

In [ ]:
# Cell with the code to display the content of the selected SDS file
import os
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from sdsconvert import RecordManager
import matplotlib.pyplot as plt

# Function to display the content of the selected file
def display_file_content(change):
    selected_filename = change['new']
    if selected_filename == "Select SDS File...":
        return
    
    selected_filepath = sds_files[sds_filenames.index(selected_filename) - 1]
    
    # Create RecordManager instance
    record_mgr = RecordManager()

    # Open and read the SDS file
    with open(selected_filepath, 'rb') as file:
        data = record_mgr.getData(file)

    # Extract timestamps and convert to seconds
    timestamps = np.array(data['timestamp']) / 1000.0  # Convert ms to s

    # Get raw data and convert according to the data format
    raw_data = data['raw_data']
    data_size = data['data_size']

    # Process the raw data into numpy arrays
    gyro_data = []
    offset = 0
    for size in data_size:
        # Each sample has 3 values (x,y,z) of 2 bytes each
        sample = np.frombuffer(raw_data[offset:offset+6], dtype=np.int16)
        gyro_data.append(sample)
        offset += size

    gyro_data = np.array(gyro_data)

    # Plot the data
    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, gyro_data[:, 0], label='X-axis')
    plt.plot(timestamps, gyro_data[:, 1], label='Y-axis')
    plt.plot(timestamps, gyro_data[:, 2], label='Z-axis')
    plt.xlabel('Time (s)')
    plt.ylabel('Gyroscope Reading')
    plt.title(f'Gyroscope Data from {selected_filename}')
    plt.legend()
    plt.grid(True)
    plt.show()



# Visualize SDS Data

The provided code snippet is a Jupyter Notebook cell that allows users to select and visualize data from .sds files using a dropdown. Run the block to populate the dropdown menu with the files in the folder Test.

In [ ]:
# Run this cell to find SDS files and update dropdown
sds_files = []
sds_filenames = ["Select SDS File..."]

# Walk through the directory
for root, dirs, files in os.walk('..'):
    for file in files:
        if file.endswith('.sds'):
            sds_files.append(os.path.join(root, file))
            sds_filenames.append(file)

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=sds_filenames,
    description='SDS Files:',
    disabled=False,
)

# Attach the callback function to the dropdown
dropdown.observe(display_file_content, names='value')

# Display the dropdown
display(dropdown)

